In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from time import sleep
import json, os

# get page

In [2]:
driver = webdriver.Chrome()

In [3]:
url = 'https://pantip.com/topic/42367932'
driver.get(url)

# get post content

In [4]:
soup = BeautifulSoup(driver.page_source)
main_post = soup.select_one('div.main-post-inner')

title = main_post.h2.text.strip()
tags = [tag.text.strip() for tag in main_post.select('a.tag-item')]
text = main_post.select_one('div.display-post-story').get_text(separator='\n').strip()
profile = main_post.select_one('div.display-post-avatar').a.get('href')
time =  main_post.select_one('div.display-post-avatar').abbr.get('data-utime')

{
    'title': title,
    'tags': tags,
    'text': text,
    'profile': profile,
    'time': time
}

{'title': 'อยากจบชีวิตตัวเอง เพราะชีวิตเหมือนตายทั้งเป็น',
 'tags': ['ปัญหาชีวิต', 'สุขภาพจิต', 'โรคซึมเศร้า'],
 'text': 'ผิดไหม? ถ้าเราอยากจากโลกนี้ไปตลอดกาล...\n\nขอโทษที่เราต้องตั้งกระทู้ที่สิ้นคิดแบบนี้ ไม่คิดเหมือนกันว่าตัวเองจะมาถึงจุดนี้ หมดแรงที่จะสู้ต่อไปแล้ว ภายนอกพังยับเยิน ภายในแทบไม่เหลือชิ้นดี',
 'profile': 'https://pantip.com/profile/7860906',
 'time': '11/27/2023 13:47:44'}

In [5]:
# make function
def get_one_post(soup):
    main_post = soup.select_one('div.main-post-inner')

    title = main_post.h2.text.strip()
    tags = [tag.text.strip() for tag in main_post.select('a.tag-item')]
    text = main_post.select_one('div.display-post-story').get_text(separator='\n').strip()
    profile = main_post.select_one('div.display-post-avatar').a.get('href')
    time =  main_post.select_one('div.display-post-avatar').abbr.get('data-utime')

    return {
        'title': title,
        'tags': tags,
        'text': text,
        'profile': profile,
        'time': time
    }

# for loop

In [8]:
df = pd.read_csv('../output - wip/df_pantip_depression_post_url.csv')
df.drop(index=[1], inplace=True)

url_to_scrape = df['0']

url_to_scrape[:10]

0     https://pantip.com/topic/42367932
2     https://pantip.com/topic/42367590
3     https://pantip.com/topic/42367222
4     https://pantip.com/topic/42367045
5     https://pantip.com/topic/42366704
6     https://pantip.com/topic/42366678
7     https://pantip.com/topic/42366406
8     https://pantip.com/topic/42365976
9     https://pantip.com/topic/42365939
10    https://pantip.com/topic/42365259
Name: 0, dtype: object

In [3]:
filename = 'pantip_suicide.json'

## open scraped file
if os.path.exists(filename):
    with open(filename, 'r', encoding='utf8') as f:
        data = json.load(f)
else:
    data = []

## make unscraped urls
if len(data) != 0:
    scraped_url = [dic['url'] for dic in data]
    url_to_scrape = sorted(set(url_to_scrape) - set(scraped_url)) # set difference

print('Num of URL :', len(url_to_scrape))

## continue to scrape
for url in tqdm(url_to_scrape[:5000]):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source)
        dic = get_one_post(soup)
        dic['url'] = url
        data.append(dic)

        # save as json file
        with open('pantip_suicide.json', 'w', encoding='utf8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
    except:
        pass

Num of URL : 24100


100%|██████████| 5000/5000 [00:00<?, ?it/s]


# open file

In [15]:
pd.read_json('../output/pantip_suicide.json')

,title,tags,text,profile,time,url
0,อยากจบชีวิตตัวเอง เพราะชีวิตเหมือนตายทั้งเป็น,"[ปัญหาชีวิต, สุขภาพจิต, โรคซึมเศร้า]",ผิดไหม? ถ้าเราอยากจากโลกนี้ไปตลอดกาล...\n\nขอโ...,https://pantip.com/profile/7860906,11/27/2023 13:47:44,https://pantip.com/topic/42367932
1,ความคิด,"[โรคย้ำคิดย้ำทำ, สุขภาพจิต, โรคซึมเศร้า]",ตอนนี้รู้สึกเบื่อจากที่เคยชอบก็ไม่ชอบแล้ว รู้ส...,https://pantip.com/profile/7860635,11/27/2023 10:36:11,https://pantip.com/topic/42367590
2,ขอวิธีหลุดพ้นจากคนที่ไม่ได้รัก,"[โรคซึมเศร้า, ศาลาคนเศร้า]",เรื่องคือ เราคบกับเขาได้2-3ปีเเล้วตอนนี้ เรารั...,https://pantip.com/profile/7860394,11/27/2023 03:43:29,https://pantip.com/topic/42367222
3,อาการแบบนี้เข้าข่ายโรคซึมเศร้ามั้ยครับ,"[โรคซึมเศร้า, โรคย้ำคิดย้ำทำ, สุขภาพจิต, จิตแพ...",ตามหัวกระทู้เลยครับ\n\n\n\nขอเกริ่นก่อนว่าตอนน...,https://pantip.com/profile/7860223,11/27/2023 00:09:00,https://pantip.com/topic/42367045
4,เราไม่รู้ว่าเราเป็นโรคซึมเศร้ามั้ย,[โรคซึมเศร้า],แต่อาการเราคล้ายมากไม่รู้คิดเองไปหรือเปล่า อาก...,https://pantip.com/profile/7859934,11/26/2023 20:41:24,https://pantip.com/topic/42366704
...,...,...,...,...,...,...
24087,เครียดจนกินไม่ได้,"[โภชนาการ, โรคซึมเศร้า, สุขภาพจิต, จิตแพทย์]",มีใครเครียดจนกินไม่ได้บ้างคะ คือกินไม่ลงจิงๆวั...,https://pantip.com/profile/7686765,11/23/2023 10:51:32,https://pantip.com/topic/42359912
24088,เป็นซึมเศร้า รู้สึกว่าตัวเองล้มเหลวในเรื่องการ...,"[โรคซึมเศร้า, สุขภาพจิต, ปัญหาชีวิต]",เราเคยทำงานบ.ที่ขึ้นชื่อเรื่องความกดดัน 2 ตำแห...,https://pantip.com/profile/7735837,11/23/2023 14:35:23,https://pantip.com/topic/42360378
24089,อยู่ม.2เทอม2 ย้ายที่เรียนได้ไหมคะ,"[โรคซึมเศร้า, การศึกษานอกโรงเรียน (กศน.), การเ...",สวัสดีค่า หนูชื่ออลันนะคะ อายุ14 ตามหัวข้อเลยค...,https://pantip.com/profile/7854615,11/23/2023 15:15:56,https://pantip.com/topic/42360444
24090,ทำไมเมื่ออายุมากขึ้น ความสุขในชีวิต ยิ่งน้อยลง,"[ปัญหาชีวิต, สุขภาพจิต, โรคซึมเศร้า, จิตวิทยา ...",ทำไมเมื่ออายุมากขึ้น ความสุขในชีวิต ยิ่งน้อยลง...,https://pantip.com/profile/2414611,11/23/2023 20:15:21,https://pantip.com/topic/42361003


In [16]:
df_pantip_depression_post = pd.read_json('../output/pantip_suicide.json')

In [17]:
df_pantip_depression_post.duplicated(subset = ['url']).any()

False

In [18]:
# Define the list of keywords to search for in the 'title' or 'text' column
keywords = ['suicide', 'suicides', 'suicided', 'suicidal', 'suici',
            'harm',	'harms', 'harmed',
            'อยากฆตต', "ฆตต",
            "อยากฆ่าตัวตาย", "ฆ่าตัวตาย",
            "อยากค่าตัวตาย", "ค่าตัวตาย",
            "อยากจบชีวิต", "จบชีวิต", 
            "อยากสิ้นชีวิต", "สิ้นชีวิต",
            "อยากคิดสั้น", "คิดสั้น",
            "อยากโดดตึก", "โดดตึก",
            'อยากโดดน้ำ',	'โดดน้ำ',
            'อยากโดดสะพาน',	'โดดสะพาน',
            'อยากโดดที่สูง',	'โดดที่สูง',
            'อยากโดดจากที่สูง',	'โดดจากที่สูง',
            'อยากโดดให้รถชน',	'โดดให้รถชน',
            "อยากแขวนคอ", "แขวนคอ",
            "อยากกินยาพิษ", "กินยาพิษ",
            "อยากกินยาเกินขนาด", "กินยาเกินขนาด",
            'อยากกินยาฆ่าแมลง',	'กินยาฆ่าแมลง',
            'อยากกินยาเบือหนู',	'กินยาเบื่อหนู',
            'อยากยิงตัวตาย',	'ยิงตัวตาย',
            'อยากยิงหัวตัวเอง',	'ยิงหัวตัวเอง',
            'อยากกรีดข้อมือ',	'กรีดข้อมือ',
            'อยากทำร้ายตัวเอง',	'ทำร้ายตัวเอง',
            'อยากตายบ่อย',	'อยากตายตลอด',	'อยากตายทุกวัน',	'อยากตายมาก',
            
            "อยากหายไปจากโลก", "หายไปจากโลก", 
            "อยากหายจากโลก", "หายจากโลก", 
            "อยากหายตัวไป", "หายตัวไป",
            "อยากหายไป", "หายไป",
            'อยากหายจากบ้าน',	'หายจากบ้าน',
            "ไม่อยากมีชีวิต", "ไม่มีชีวิต", 
            "ไม่อยากตื่น", "ไม่ตื่น", 
            'อยากนอนไปตลอด', 'นอนไปตลอด',
            'อยากนอนตลอดไป', 'นอนตลอดไป',
            "ไม่อยากอยู่ต่อ", "ไม่อยู่ต่อ", 
            'ไม่อยากอยู่บนโลก',	'ไม่อยู่บนโลก',
            'ไม่อยากอยู่',	'ไม่อยู่',
            'อยากหนีไปจากโลก',	'หนีไปจากโลก',
            'อยากหนีไปจากบ้าน',	'หนีไปจากบ้าน',
            'อยากหนีไป', 'หนีไป',
            'อยากหนี',	'หนี',
            'อยากให้โลกแตก', 'โลกแตก',
            'อยากให้โลกหายไป',	'โลกหายไป',
            'อยากให้โลกจบสิ้น',	'โลกจบสิ้น',
            'อยากให้รถชนตาย', 'อยากให้รถชน',
            "อยากตาย", "ตาย",
            'escape',	'escapes',	'escaped',		
            'die',	'dies',	'died',	'dying', 'dead',	'death',  'deaths'
            ]

# Filter rows containing the keywords in the 'title' column
df_pantip_suicidal_post_title = df_pantip_depression_post[df_pantip_depression_post['title'].str.contains('|'.join(keywords))]

# Count the number of rows in the filtered DataFrame
count_of_matching_rows = len(df_pantip_suicidal_post_title)

# Display the count and the df_pantip_suicidal_post DataFrame
print(f"Number of rows containing the keywords: {count_of_matching_rows}")
print(df_pantip_suicidal_post_title)

Number of rows containing the keywords: 2319
                                                   title  \
0          อยากจบชีวิตตัวเอง เพราะชีวิตเหมือนตายทั้งเป็น   
5      แม่พูดย้ำๆให้ผมไปตายเพื่อชดใช้ความผิด ผมควรเลื...   
10     ช่วยให้คำเเนะนำรุ่นน้อง ที่อยากฆ่าตัวตายหน่อยครับ   
13                                       เมื่อคุณอยากตาย   
42     ขี้ลืมมากความจำสั้นโคตรๆเฉื่อยช้ามากไม่มีชีวิต...   
...                                                  ...   
24021               โรคซึมเศร้า ไม่อยากมีชีวิตอยู่ต่อ...   
24029                          ไม่อยากมีชีวิตอยู่แล้วล่ะ   
24039              เราไม่มีอะไรยึดเหนี่ยวในการมีชีวิตต่อ   
24049  จำเป็นไหมครับ คนที่คิดสั้นฆ่าตัวตายจะต้องเป็นโ...   
24057         มีความรู้สึกที่ไม่อยากมีชีวิตอยู่ต่อไปแล้ว   

                                                    tags  \
0                   [ปัญหาชีวิต, สุขภาพจิต, โรคซึมเศร้า]   
5      [โรคซึมเศร้า, สุขภาพจิต, กฎหมายชาวบ้าน, การเงิ...   
10     [โรคซึมเศร้า, สุขภาพจิต, ปัญหาชีวิต, จิตแพทย์,.

In [19]:

# Filter rows containing the keywords in the 'text' column
df_pantip_suicidal_post_text = df_pantip_depression_post[df_pantip_depression_post['text'].str.contains('|'.join(keywords))]

# Count the number of rows in the filtered DataFrame
count_of_matching_rows = len(df_pantip_suicidal_post_text)

# Display the count and the df_pantip_suicidal_post DataFrame
print(f"Number of rows containing the keywords: {count_of_matching_rows}")
print(df_pantip_suicidal_post_text)

Number of rows containing the keywords: 10776
                                                   title  \
3                 อาการแบบนี้เข้าข่ายโรคซึมเศร้ามั้ยครับ   
4                     เราไม่รู้ว่าเราเป็นโรคซึมเศร้ามั้ย   
5      แม่พูดย้ำๆให้ผมไปตายเพื่อชดใช้ความผิด ผมควรเลื...   
6      อกหักจนเป็นซึมเศร้า อยากรู้แต่ละคนผ่านช่วงเวลา...   
9                           ทำใจยังไงให้ได้จากการสูญเสีย   
...                                                  ...   
24076  (ชวนคุย) ทำไมการุณยฆาตถึงถูกพูดแต่ในแง่ของความ...   
24078  โรคใหม่ -Post Vaccine Syndrome- งานศึกษาพบทำให...   
24079  เมื่อคนที่เรารักมากๆ ป่วยหนัก เรามีวิธีจัดการก...   
24081  ขอสอบถามได้ไหมคะ เราสงสัยว่าเราเป็นซึมเศร้าขั้...   
24091  อยากรู้ว่าคัวเองเข้าค่ายคนเป็นโรคซึมเศร้ารึยังคะ?   

                                                    tags  \
3      [โรคซึมเศร้า, โรคย้ำคิดย้ำทำ, สุขภาพจิต, จิตแพ...   
4                                          [โรคซึมเศร้า]   
5      [โรคซึมเศร้า, สุขภาพจิต, กฎหมายชาวบ้าน, การเงิ

In [20]:
# identify the number of rows which have the same url between df_pantip_suicidal_post_title and df_pantip_suicidal_post_text
df_pantip_suicidal_post_title['url'].isin(df_pantip_suicidal_post_text['url']).value_counts()

url
True     1824
False     495
Name: count, dtype: int64

In [21]:
# identify the number of rows which have the same url between df_pantip_suicidal_post_title and df_pantip_suicidal_post_text
df_pantip_suicidal_post_text['url'].isin(df_pantip_suicidal_post_title['url']).value_counts()

url
False    8952
True     1824
Name: count, dtype: int64

In [22]:
# crate the new dataframe which contain the rows of df_pantip_suicidal_post_text only
df_pantip_suicidal_post_text_only = df_pantip_suicidal_post_text[~df_pantip_suicidal_post_text['url'].isin(df_pantip_suicidal_post_title['url'])]
# check shape of the new dataframe
df_pantip_suicidal_post_text_only.shape

(8952, 6)

In [37]:
# save the dataframe which based on title portion to output - wip folder (for analysis to do labeling only)
df_pantip_suicidal_post_title.to_csv('../output - wip/df_pantip_suicidal_post_title.csv', index = True, encoding = 'UTF8')

In [38]:
# save the dataframe which based on text portion to output - wip folder (for analysis to do labeling only)
df_pantip_suicidal_post_text.to_csv('../output - wip/df_pantip_suicidal_post_text.csv', index = True, encoding = 'UTF8')

In [39]:
# save the dataframe which based on text only portion to output - wip folder (for analysis to do labeling only)
df_pantip_suicidal_post_text_only.to_csv('../output - wip/df_pantip_suicidal_post_text_only.csv', index = True, encoding = 'UTF8')  

In [6]:
# save the all post dataframe to output - wip folder (for analysis to do labeling only)
df_pantip_depression_post.to_csv('../output - wip/df_pantip_depression_post.csv', index = True, encoding = 'UTF8')